#### Importing libraries

In [1]:
import xlwings as xw
from datetime import datetime
import openpyxl
from openpyxl.utils import get_column_letter
import numpy as np
import re
import pandas as pd


##### Opening files

In [2]:
#Reading files
user = input("Enter your laptop user: ")
# Get the current date
current_date = datetime.now().strftime("%Y%m%d")

gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day}.xlsx'.format(user=user, çurrent_day=current_date))
sec_dep_det = xw.Book(r'C:\Users\{user}\Downloads\security_deposit_funds_detail-{çurrent_day}.xlsx'.format(user=user,çurrent_day=current_date))

# Indicating worksheet
gl_worksheet = gl.sheets['Sheet1']
sec_dep_det_worksheet = sec_dep_det.sheets['Sheet1']

#### Diagnostics 1

In [3]:
# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])



In [4]:
# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 



True

#### Notes

No cambiarle el nombre a los archivos que se descargan de appfolio